# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

## This is a WIP: Decided to do omdb data cleaning first - bevore a useful model can be applied

Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [2]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [3]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [4]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [5]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [6]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [7]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [8]:
# Select relevant columns
omdb = omdb[['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_month', 'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated']]

In [9]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      9
Language                     0
imdbRating                   4
imdbVotes                    5
Rotten Tomatoes           1237
Metacritic                4019
Series                       0
Released_month              56
PG_Rating                 2314
Available_languages          0
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [10]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0:'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [11]:
# missing entries in omdb dataset
omdb[omdb['Year'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [12]:
omdb.isna().sum()

imdbID                       0
Year                        10
Runtime                     15
Language                    10
imdbRating                  10
imdbVotes                   11
Rotten Tomatoes           1246
Metacritic                4025
Series                      10
Released_month              62
PG_Rating                 2320
Available_languages         10
Oscars_won                  10
Oscars_nominated            10
Golden_globe_won            10
Golden_globe_nominated      10
dtype: int64

In [13]:
# Replace NaN with 0 and handle accordingly for awards
for i in range (9,13):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [14]:
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)


In [15]:
# Comment Max: Vorschlag - Stattdessen Median/Modalwert benutzen benutzen
# Fill NaN Series
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)
# Fill NaN Runtime
omdb.loc[:,'Runtime'] = omdb.loc[:,'Runtime'].fillna(omdb.loc[:,'Runtime'].median())
# Fill median for all others
for i in range (4,16):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(omdb.iloc[:,i].median())

In [16]:
merged_data = ratings.merge(omdb, how='left', on='imdbID')

In [17]:
merged_data.head()

,Unnamed: 0,user_id,imdbID,rating,Year,Runtime,Language,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,1264,tt0047034,3.5,1954.0,96.0,['Japanese'],7.6,27485.0,9.3,7.8,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,213,tt0304141,2.5,2004.0,142.0,['English'],7.9,524511.0,9.0,8.2,0.0,6.0,1.0,1.0,0.0,2.0,0.0,0.0
2,2,593,tt0369436,3.0,2008.0,88.0,['English'],5.7,61335.0,2.4,4.1,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,609,tt1077258,4.0,2007.0,105.0,['English'],7.1,196149.0,7.4,5.9,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,1590,tt0052182,4.0,1958.0,100.0,['English'],7.4,6337.0,6.9,5.9,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0


In [18]:
merged_data = merged_data.drop(columns={'Unnamed: 0'})

In [19]:
# Comment Max: No NaN rows anymore - except for language for the missing movies
merged_data.isna().sum()

user_id                     0
imdbID                      0
rating                      0
Year                      269
Runtime                     0
Language                  269
imdbRating                  0
imdbVotes                   0
Rotten Tomatoes             0
Metacritic                  0
Series                      0
Released_month              0
PG_Rating                   0
Available_languages         0
Oscars_won                  0
Oscars_nominated            0
Golden_globe_won            0
Golden_globe_nominated      0
dtype: int64

In [20]:
merged_data[merged_data['Language'].isna()]

,user_id,imdbID,rating,Year,Runtime,Language,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
1349,1774,tt0800175,3.0,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3403,1206,tt0800175,0.5,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4169,226,tt0720339,3.0,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9942,393,tt0439663,3.5,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15220,612,tt0257001,2.5,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781823,59,tt0800175,1.0,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
783363,800,tt0266860,3.5,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784056,538,tt0257001,3.0,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784667,1128,tt0266860,4.0,NaN,101.0,NaN,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#Drop NaN rows - check later if it makes large difference!
merged_data = merged_data.dropna()

In [22]:
merged_data = merged_data.set_index('imdbID').reset_index()

In [23]:
# possibility to encode language as one-hot encoding
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(merged_data['Language'])

In [24]:
# however, do not include language yet
merged_data = merged_data.drop(columns={'Language'})

In [25]:
data_as_array = merged_data.to_numpy()

In [26]:
le = preprocessing.LabelEncoder()
for i in range(0,17):
    data_as_array[:,i] = le.fit_transform(data_as_array[:,i])

In [27]:
csr_data = csr_matrix(data_as_array.astype(np.float))
merged_data.reset_index(inplace=True)

In [28]:
k_neighbors = 10
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

In [29]:
print("Training")
knn.fit(csr_data)

Training


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [30]:
csr_data

<787272x17 sparse matrix of type '<class 'numpy.float64'>'
	with 9268119 stored elements in Compressed Sparse Row format>

In [31]:
# Do predictions
print("Do Prediction")
movie_idx = merged_data[merged_data['imdbID'] == 'tt0113497'].index[0]
print(movie_idx)
distances, indices = knn.kneighbors(csr_data[movie_idx], n_neighbors=k_neighbors+1)
print(distances)
print(indices)

Do Prediction
3198
[[2.22044605e-16 1.18844555e-08 2.37685370e-08 2.37686324e-08
  5.94218412e-08 1.01016062e-07 1.48555022e-07 1.72321494e-07
  2.19860632e-07 4.04063544e-07 4.04069806e-07]]
[[  3198 619623 237227 429293 343688 331914 696466 397476  17977 621186
   84772]]


In [32]:
for i in range(0,indices.size):
    print(merged_data.iloc[[indices[0,i]]])

      index     imdbID  user_id  rating    Year  Runtime  imdbRating  \
3198   3198  tt0113497       42     3.0  1995.0    104.0         7.0   

      imdbVotes  Rotten Tomatoes  Metacritic  Series  Released_month  \
3198   294340.0              5.4         3.9     0.0            12.0   

      PG_Rating  Available_languages  Oscars_won  Oscars_nominated  \
3198        1.0                  2.0         0.0               0.0   

      Golden_globe_won  Golden_globe_nominated  
3198               0.0                     0.0  
         index     imdbID  user_id  rating    Year  Runtime  imdbRating  \
619623  619623  tt0113497       41     3.5  1995.0    104.0         7.0   

        imdbVotes  Rotten Tomatoes  Metacritic  Series  Released_month  \
619623   294340.0              5.4         3.9     0.0            12.0   

        PG_Rating  Available_languages  Oscars_won  Oscars_nominated  \
619623        1.0                  2.0         0.0               0.0   

        Golden_globe_won  

In [33]:
# zip bildet tuple der beiden Objekte)
ec_movie = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
  

In [34]:
for i in ec_movie:
    print(i)

(84772, 4.0406980583718166e-07)
(621186, 4.0406354395727817e-07)
(17977, 2.1986063192258598e-07)
(397476, 1.7232149429613486e-07)
(696466, 1.4855502195043613e-07)
(331914, 1.0101606218171355e-07)
(343688, 5.9421841225315575e-08)
(429293, 2.376863239561544e-08)
(237227, 2.3768537027457626e-08)
(619623, 1.1884455530797311e-08)


In [35]:
user_ratings = merged_data

In [36]:
user_ratings

,index,imdbID,user_id,rating,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,tt0047034,1264,3.5,1954.0,96.0,7.6,27485.0,9.3,7.8,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,tt0304141,213,2.5,2004.0,142.0,7.9,524511.0,9.0,8.2,0.0,6.0,1.0,1.0,0.0,2.0,0.0,0.0
2,2,tt0369436,593,3.0,2008.0,88.0,5.7,61335.0,2.4,4.1,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,tt1077258,609,4.0,2007.0,105.0,7.1,196149.0,7.4,5.9,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,tt0052182,1590,4.0,1958.0,100.0,7.4,6337.0,6.9,5.9,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787267,787267,tt0083530,1032,3.0,1982.0,85.0,6.2,47327.0,4.2,4.8,0.0,12.0,1.0,1.0,0.0,0.0,0.0,0.0
787268,787268,tt0107798,99,3.0,1993.0,141.0,6.6,73544.0,5.3,5.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0
787269,787269,tt0093857,333,3.0,1987.0,88.0,5.1,12882.0,0.7,2.8,0.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0
787270,787270,tt0144168,49,3.0,1999.0,94.0,6.1,17926.0,7.1,6.5,0.0,8.0,2.0,1.0,0.0,0.0,0.0,0.0


In [37]:
user_ratings['imdbID'] = user_ratings['imdbID'].str.replace(r'tt','')

In [38]:
user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)

In [39]:
user_ratings

,index,imdbID,user_id,rating,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,47034.0,1264,3.5,1954.0,96.0,7.6,27485.0,9.3,7.8,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,304141.0,213,2.5,2004.0,142.0,7.9,524511.0,9.0,8.2,0.0,6.0,1.0,1.0,0.0,2.0,0.0,0.0
2,2,369436.0,593,3.0,2008.0,88.0,5.7,61335.0,2.4,4.1,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,1077258.0,609,4.0,2007.0,105.0,7.1,196149.0,7.4,5.9,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,52182.0,1590,4.0,1958.0,100.0,7.4,6337.0,6.9,5.9,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787267,787267,83530.0,1032,3.0,1982.0,85.0,6.2,47327.0,4.2,4.8,0.0,12.0,1.0,1.0,0.0,0.0,0.0,0.0
787268,787268,107798.0,99,3.0,1993.0,141.0,6.6,73544.0,5.3,5.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0
787269,787269,93857.0,333,3.0,1987.0,88.0,5.1,12882.0,0.7,2.8,0.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0
787270,787270,144168.0,49,3.0,1999.0,94.0,6.1,17926.0,7.1,6.5,0.0,8.0,2.0,1.0,0.0,0.0,0.0,0.0


In [40]:
user_ratings.dtypes

index                       int64
imdbID                    float64
user_id                     int64
rating                    float64
Year                      float64
Runtime                   float64
imdbRating                float64
imdbVotes                 float64
Rotten Tomatoes           float64
Metacritic                float64
Series                    float64
Released_month            float64
PG_Rating                 float64
Available_languages       float64
Oscars_won                float64
Oscars_nominated          float64
Golden_globe_won          float64
Golden_globe_nominated    float64
dtype: object

In [ ]:
#function that computes the k-nearest neighbors of each movie based on the features
for i in user_ratings['imdbID']:
    distances , indices = knn.kneighbors(user_ratings.loc[user_ratings['imdbID'] == i], n_neighbors=1)


In [50]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

MemoryError: Unable to allocate 426. MiB for an array with shape (71, 787272) and data type float64

In [30]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

In [37]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 10    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
     
    
    if (ratings.to_numpy().size>0):   
        
        # TODO:
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(user_ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        print('---- Training ConBF-kNN-Algorithm ----')
        print('user_id: '+str(userID))
        print('imdbID: '+str(imdbID))
        knn.fit(ratings)

        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors)
        
       
        # Collect neighbors by indices
        for i in range(0,indices.shape[1]):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbors, distances)
        
        #Generate Output for Understandability
        print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [32]:
### Testing function ###
imdbID = 113627.0
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, user_ratings)

---- Training ConBF-kNN-Algorithm ----
user_id: 123
imdbID: 113627.0
Predicted Rating for 113627.0: 4.0
Real Rating of 113627.0 was: 5.0


In [33]:
### Testing function ###
imdbID = 145653
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, user_ratings)

---- Training ConBF-kNN-Algorithm ----
user_id: 123
imdbID: 145653
Predicted Rating for 145653: 2.0
Real Rating of 145653 was: 4.5


In [34]:
## DUPLICATED RATING!!! -> durch das umschreiben mit .values[0] wird das Doppelte Rating nicht mehr angezeigt -> tdm mal checken!

In [35]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = user_ratings.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, user_ratings)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))

In [40]:
n = 80
test_predict_mr(n)

---- Training ConBF-kNN-Algorithm ----
user_id: 376.0
imdbID: 120780.0
Predicted Rating for 120780.0: 1.0
Real Rating of 120780.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 1431.0
imdbID: 94721.0
Predicted Rating for 94721.0: 4.0
Real Rating of 94721.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 337.0
imdbID: 190641.0
Predicted Rating for 190641.0: 2.5
Real Rating of 190641.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 361.0
imdbID: 94336.0
Predicted Rating for 94336.0: 4.0
Real Rating of 94336.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 340.0
imdbID: 95016.0
Predicted Rating for 95016.0: 3.5
Real Rating of 95016.0 was: 3.0
---- Training ConBF-kNN-Algorithm ----
user_id: 70.0
imdbID: 80684.0
Predicted Rating for 80684.0: 4.5
Real Rating of 80684.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 488.0
imdbID: 117119.0
Predicted Rating for 117119.0: 4.5
Real Rating of 117119.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
